In [66]:
import pandas as pd

In [67]:
drug = pd.read_csv("../data/eICU/infusiondrug.csv")
diagnosis = pd.read_csv("../data/eICU/diagnosis.csv")
patients_weight = pd.read_csv("../data/eICU/patient.csv")[["patientunitstayid", "admissionweight"]]
patient = pd.read_csv("../data/eICU/patient.csv")
drug = drug.merge(patients_weight, on="patientunitstayid")

C:\Users\eythe\AppData\Local\Temp\ipykernel_21764\3265378624.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  drug = pd.read_csv("../data/eICU/infusiondrug.csv")


In [68]:
# add hospitalID and unitType and wardID to the drug table corelated to patientunitstayid
drug["hospitalid"] = drug["patientunitstayid"].map(patient.set_index("patientunitstayid")["hospitalid"])
drug["unittype"] = drug["patientunitstayid"].map(patient.set_index("patientunitstayid")["unittype"])
drug["wardid"] = drug["patientunitstayid"].map(patient.set_index("patientunitstayid")["wardid"])

In [69]:
# filter drug from NA in drugname, patientunitstayid, hospitalid, unittype, wardid
drug = drug[~drug['drugname'].isna()]
drug = drug[~drug['patientunitstayid'].isna()]
drug = drug[~drug['hospitalid'].isna()]
drug = drug[~drug['unittype'].isna()]
drug = drug[~drug['wardid'].isna()]

#filter drug to include only drugnames that start with "Norepinephrine"
drug = drug[drug['drugname'].str.startswith("Norepinephrine")]

In [70]:
# sort drug by patientunitstayid and drugstartoffset
drug = drug.sort_values(by=["patientunitstayid", "infusionoffset"])
drug

,infusiondrugid,patientunitstayid,infusionoffset,drugname,drugrate,infusionrate,drugamount,volumeoffluid,patientweight,admissionweight,hospitalid,unittype,wardid
4,1969910,242082,55,Norepinephrine (mcg/min),2.13,8.0,4.0,250.0,NaN,90.7,108,Med-Surg ICU,136
2,1968206,242082,125,Norepinephrine (mcg/min),7,26.3,4.0,250.0,NaN,90.7,108,Med-Surg ICU,136
10,1985898,242082,230,Norepinephrine (mcg/min),9.07,34.0,4.0,250.0,NaN,90.7,108,Med-Surg ICU,136
12,1972141,242082,275,Norepinephrine (mcg/min),10,37.5,4.0,250.0,NaN,90.7,108,Med-Surg ICU,136
1,1998443,242082,425,Norepinephrine (mcg/min),10.93,41.0,4.0,250.0,NaN,90.7,108,Med-Surg ICU,136
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4803535,75122092,3353251,3823,Norepinephrine MAX 32 mg Dextrose 5% 250 ml (m...,15,NaN,NaN,NaN,NaN,102.0,458,Cardiac ICU,1104
4803681,75308975,3353251,3968,Norepinephrine MAX 32 mg Dextrose 5% 250 ml (m...,16,NaN,NaN,NaN,NaN,102.0,458,Cardiac ICU,1104
4803654,75246300,3353251,4043,Norepinephrine MAX 32 mg Dextrose 5% 250 ml (m...,12,NaN,NaN,NaN,NaN,102.0,458,Cardiac ICU,1104
4803592,75148203,3353251,4358,Norepinephrine MAX 32 mg Dextrose 5% 250 ml (m...,10,NaN,NaN,NaN,NaN,102.0,458,Cardiac ICU,1104


In [9]:
drug.groupby(["hospitalid", "unittype"])["patientunitstayid"].nunique().sort_values()

hospitalid  unittype    
342         Med-Surg ICU      1
151         Med-Surg ICU      1
265         MICU              1
156         Med-Surg ICU      1
438         Med-Surg ICU      1
                           ... 
157         Med-Surg ICU    375
167         Med-Surg ICU    384
188         MICU            430
300         Med-Surg ICU    452
199         Med-Surg ICU    525
Name: patientunitstayid, Length: 195, dtype: int64

In [114]:
bp = pd.read_csv("../preprocess/filtered_bp_eicu.csv")
bp = bp.sort_values(by=["stay_id", "cur_bp_time"])
bp = bp[bp["stay_id"].isin(drug["patientunitstayid"])]
bp["next_bp_time"] = bp.groupby("stay_id")["cur_bp_time"].shift(-1)
bp["interval"] = bp["next_bp_time"] - bp["cur_bp_time"]
drug["drugrate"] = pd.to_numeric(drug["drugrate"], errors='coerce')
bp["age"] = pd.to_numeric(bp["stay_id"].map(patient.set_index("patientunitstayid")["age"]), errors='coerce')
bp["hospitalid"] = bp["stay_id"].map(patient.set_index("patientunitstayid")["hospitalid"])
bp["unittype"] = bp["stay_id"].map(patient.set_index("patientunitstayid")["unittype"])
bp["wardid"] = bp["stay_id"].map(patient.set_index("patientunitstayid")["wardid"])
# create a new column for the number of patients per hospital
bp["pat_per_hos"] = bp["hospitalid"].map(bp.groupby(["hospitalid"]).agg({"stay_id": ["nunique"]}).sort_values(by=("stay_id", "nunique")))


ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [112]:
# find the number of patients per hospital
bp.groupby(["hospitalid"]).agg({"stay_id": ["nunique"]}).sort_values(by=("stay_id", "nunique"))


,stay_id
,nunique
hospitalid,
279,1
422,1
174,1
404,1
182,1
...,...
157,155
148,156


In [ ]:
bp["pat_per_hos"]

In [44]:
# give me a table of hospitals, wardid and number of patients per hospital
bp_hospital = bp.groupby(['hospitalid']).agg({'stay_id': ['nunique']})

In [62]:
bp["stay_id"].unique()

array([ 243334,  243643,  244256, ..., 3352721, 3353144, 3353251],
      dtype=int64)

In [28]:
print(f'Number of patients in the dataset: {bp_big["stay_id"].nunique()}')
print(f'Number of hospitals in the dataset: {bp_big["hospitalid"].nunique()}')
print(f'Number of patients per hospital in the dataset: {bp_big["pat_per_hos"].unique()}')

Number of patients in the dataset: 2274
Number of hospitals in the dataset: 23
Number of patients per hospital in the dataset: [ 45 199 144 238 156 155 120  44 138 114 133  80  23   2   3   1  63  38
 342  76  37]
